In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import datetime
import re
import time
import urllib
import pandas as pd
import numpy as np

url = "https://cointelegraph.com/tags/bitcoin-regulation"

In [2]:
driver = webdriver.Firefox()
driver.get(url)
html = driver.execute_script("return document.documentElement.outerHTML")
selsoup = BeautifulSoup(html, "html.parser")

In [44]:
for titlesoup in selsoup.findAll("h2",{"class":"header"}):
    for link2 in titlesoup.findAll("a", href = re.compile("^(https://cointelegraph.com/news/)")):
        if 'href' in link2.attrs:
            linkname = link2.attrs['href']
#         print(linkname)

https://cointelegraph.com/news/canada-releases-official-draft-of-new-crypto-regulations-focused-on-kyc-aml
https://cointelegraph.com/news/colombian-banks-close-all-accounts-of-s-american-crypto-exchange-buda
https://cointelegraph.com/news/china-as-a-prime-example-of-blockchain-before-bitcoin-policy
https://cointelegraph.com/news/wsj-cme-and-btc-exchange-dispute-led-us-regulators-to-open-price-manipulation-probe
https://cointelegraph.com/news/united-states-v-crypto-etfs-a-lingering-struggle-for-mass-adoption
https://cointelegraph.com/news/governor-of-connecticut-signs-blockchain-working-group-bill-into-law
https://cointelegraph.com/news/quebec-halts-crypto-mining-electricity-requests-amid-huge-demand-forms-new-framework
https://cointelegraph.com/news/polish-financial-authority-says-crypto-trading-is-legal-in-wake-of-anti-crypto-campaign
https://cointelegraph.com/news/sec-us-crypto-exchanges-not-enthusiastic-enough-about-regulatory-compliance
https://cointelegraph.com/news/coinbase-acqui

In [ ]:
# scraping recent news articles

In [4]:
driver = webdriver.Firefox()
driver.get(url)
for i in range(1,100):
    elem = driver.find_element_by_link_text("Load more articles")
    elem.send_keys(Keys.RETURN)
    
#     for link in selsoup.findAll("a", href = re.compile("^(https://cointelegraph.com/news/)")):
#         if 'href' in link.attrs:
#             linkname = link.attrs['href']
#         print(link.attrs['href'])
#         linkno=linkno+1
#             f.write(linkname + "\n")
#     f.close()
    
    i += 1
time.sleep(10)
# f.close()

In [83]:
html = driver.page_source
mysoup = BeautifulSoup(html, "html.parser")

In [84]:
filename = "recentlinks.csv"
f = open(filename, 'w')

headers = "linkname\n"

f.write(headers)

9

In [85]:
for link in mysoup.findAll("a", href = re.compile("^(https://cointelegraph.com/news/)")):
    if 'href' in link.attrs:
        linkname = link.attrs['href']
#         print(link.attrs['href'])
#         linkno=linkno+1
        f.write(linkname + "\n")
#     f.close()
    
#     i += 1
f.close()

In [ ]:
# now scraping top news articles

In [15]:
driver = webdriver.Firefox()
driver.get(url)

topnews = driver.find_element_by_link_text("Top News")
topnews.send_keys(Keys.RETURN)

for i in range(1,100):
    elem = driver.find_element_by_link_text("Load more articles")
    elem.send_keys(Keys.RETURN)
     
    i += 1
time.sleep(10)

In [16]:
html = driver.page_source
mysoup = BeautifulSoup(html, "html.parser")

In [17]:
filename = "toplinks.csv"
f = open(filename, 'w')

headers = "linkname\n"

f.write(headers)

9

In [18]:
for link in mysoup.findAll("a", href = re.compile("^(https://cointelegraph.com/news/)")):
    if 'href' in link.attrs:
        linkname = link.attrs['href']

        f.write(linkname + "\n")

f.close()

In [19]:
# now scrape the most commented articles

In [20]:
driver = webdriver.Firefox()
driver.get(url)

topnews = driver.find_element_by_link_text("Commented")
topnews.send_keys(Keys.RETURN)

for i in range(1,100):
    elem = driver.find_element_by_link_text("Load more articles")
    elem.send_keys(Keys.RETURN)
     
    i += 1
time.sleep(10)

In [21]:
html = driver.page_source
mysoup = BeautifulSoup(html, "html.parser")

In [22]:
filename = "commented.csv"
f = open(filename, 'w')

headers = "linkname\n"

f.write(headers)

9

In [23]:
for link in mysoup.findAll("a", href = re.compile("^(https://cointelegraph.com/news/)")):
    if 'href' in link.attrs:
        linkname = link.attrs['href']

        f.write(linkname + "\n")

f.close()

In [46]:
recentlinks = pd.read_csv("ctlinks.csv")
toplinks = pd.read_csv("toplinks.csv")
commentedlinks = pd.read_csv("commented.csv")

In [55]:
recentlinks = recentlinks.drop_duplicates(subset=['linkname'], keep='first')

In [57]:
recentlinks = recentlinks.reset_index(drop=True)

In [141]:
frames = [recentlinks, toplinks, commentedlinks]
linksall = pd.concat(frames)

In [143]:
linksall = linksall.drop_duplicates(subset=['linkname'], keep='first')

In [145]:
linksall = linksall[linksall.linkname != 'linkname']

In [148]:
# linksall.columns = [''] * len(linksall.columns)

In [152]:
linksall['linkname'] = linksall['linkname'].str.strip()

In [154]:
linksall['linkname'] = linksall['linkname'].str.replace(" ","")

In [162]:
linksall.iloc[1][0]

'https://cointelegraph.com/news/japan-s-financial-watchdog-rejects-crypto-exchange-license-for-first-time'

In [166]:
linksall.to_csv("linksall.csv", encoding='utf-8', index=False)

In [229]:
%%time

title=[]
author=[]
dt=[]
views=[]
shares=[]
abody=[]
comments=[]

for link in linksall.linkname:
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    soupa = soup.find(class_="name")
    soupt = soup.find(class_="header")
    soupd = soup.find(class_="date")
    soupv = soup.find(class_="referral_stats total-views")
    soups = soup.find(class_="referral_stats total-shares")
    soupb = soup.find(class_="post-full-text contents js-post-full-text")
    soupc = soup.find(class_="comments-count")
    text =""
    texts = soupb.find_all("p")
    for row in texts:
        text += row.get_text()

    author1 = soupa.text.strip()
    title1 = soupt.text.strip()
    dt1 = soupd['datetime']
    views1 = soupv.find(class_="total-qty").text.strip()
    shares1 = soups.find(class_="total-qty").text.strip()
    abody1 = text

    try:
        comments1 = soupc.text.strip()
    except Exception:
        comments1 = []
    title.append(title1)
    author.append(author1)
    dt.append(dt1)
    views.append(views1)
    shares.append(shares1)
    abody.append(abody1)
    comments.append(comments1)

Wall time: 1h 30min 59s


In [230]:
df = pd.DataFrame()
df = df.fillna(0) # with 0s rather than NaNs

In [233]:
df['title'] = title
df['author'] = author
df['dt'] = dt
df['views'] = views
df['shares'] = shares
df['abody'] = abody
df['comments'] = comments

In [234]:
df.head(10)

,title,author,dt,views,shares,abody,comments
0,SEC Chairman Jay Clayton Says Bitcoin Not a Se...,Molly Jane Zuckerman,2018-06-07 16:05:19,21383,422,"Jay Clayton, the chair of the US Securities an...",5
1,Japan's Financial Watchdog Rejects Crypto Exch...,Marie Huillet,2018-06-07 13:15:18,8146,158,Japan’s Financial Services Agency (FSA) has ba...,[]
2,LexisNexis Partners With Australian Crypto Exc...,Helen Partz,2018-06-05 13:45:18,18990,205,Risk management giant LexisNexis has partnered...,[]
3,Russian Court Annuls Previous Decision to Bloc...,Helen Partz,2018-06-05 12:15:14,10605,211,"The City Court of St. Petersburg, Russia has o...",[]
4,Former SEC Chair Represents Ripple in Lawsuit ...,Ana Alexandre,2018-06-05 12:56:15,14094,172,Fintech company Ripple Labs has employed Debev...,1
5,SEC Names New Senior Advisor to Oversee Crypto...,Marie Huillet,2018-06-05 20:05:25,9216,192,The U.S. Securities and Exchange Commission (S...,[]
6,Former US Federal Prosecutor at Money20/20: ‘A...,Marie Huillet,2018-06-05 06:19:57,7957,321,Former U.S. Federal Prosecutor Mary Beth Bucha...,[]
7,Spain Introduces Innovation-Aimed Crypto Regul...,Stephen O'Neal,2018-06-05 05:45:40,13978,285,"On May 30, the Spanish Congress unanimously su...",8
8,New York State Legislature Proposes Creation o...,Ana Alexandre,2018-06-04 13:20:17,22454,263,The banks committee of the New York state legi...,1
9,Japan: Financial Watchdog Probes Crypto Exchan...,Marie Huillet,2018-06-02 08:27:47,17063,165,Japan’s Financial Services Agency (FSA) has is...,11


In [235]:
df.to_csv("df.csv",index=None)